In [1]:
import os
import sys
import pandas as pd
from snp_query_box import hraQueries
user_id = "A845514"
os.getcwd()
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(f'{parent_dir}/dsnp_transform')
import hraResultTransform as hrt


In [2]:
pull_hra_qa_by_section_services_and_support = hraQueries.pull_hra_qa_by_section_bq(asad_section_nm = 'Services and Support')

#path = r"C:\Users\A845514\Projects_code\data\hra_result\pull_db"
#pull_hra_qa_by_section_pain = pd.read_pickle(f'{path}/pull_hra_qa_by_section_pain.pkl')

asm_id_value_desc
asad_template_nm
asad_template_version_nbr
asad_page_nm
asad_assmnt_tab_id
asad_tab_nm
asad_assmnt_section_id
asad_section_nm
asad_assmnt_status_typ_key_cd
asad_assmnt_question_id
asad_question_txt
asad_updated_on_dts
asad_assmnt_ans_id
asad_assmnt_ans_typ_desc
asad_assmnt_ans_data_txt


In [3]:
hra_service_and_support = (pull_hra_qa_by_section_services_and_support.pipe(hrt.get_most_recent_hra)  #possible template 3 hrt.version_three_to_four
        .pipe(hrt.pivot_hra_qa)
        .pipe(hrt.clean_service_support))

Index(['dsnp_member_id', 'any_concern', 'losing_house_concern', 'enough_food',
       'housing_situation', 'transportation_concern', 'other_concern',
       'need_help', 'asad_updated_on_dts'],
      dtype='object')


In [4]:
hra_service_and_support

,dsnp_member_id,any_concern,losing_house_concern,enough_food,housing_situation,transportation_concern,other_concern,need_help,asad_updated_on_dts,hra_section
0,,[housing],no,no,i do not have housing-staying with others,it has kept me from medical appointments|it ha...,member states that she is currently staying wi...,yes,2025-01-15 22:42:58.400000+00:00,service_and_support
1,101039985900,[i do not have a problem meeting my needs],NaN,NaN,NaN,NaN,NaN,NaN,2024-02-06 21:23:42.822122+00:00,service_and_support
2,101039986100,[i do not have a problem meeting my needs],NaN,NaN,NaN,NaN,NaN,NaN,2024-03-18 15:11:52.418199+00:00,service_and_support
3,101040086100,[i do not have a problem meeting my needs],NaN,NaN,NaN,NaN,NaN,NaN,2024-12-04 15:05:11.743333+00:00,service_and_support
4,101040408900,[i do not have a problem meeting my needs],NaN,NaN,NaN,NaN,NaN,NaN,2024-03-04 22:17:41.057540+00:00,service_and_support
...,...,...,...,...,...,...,...,...,...,...
440552,WVAQMZM00001861311,[transportation],NaN,no,NaN,it has kept me from medical appointments,NaN,yes,2024-05-15 15:13:08.776666+00:00,service_and_support
440553,WVAQMZM00001926520,"[utilities, food, ]",NaN,yes,NaN,NaN,NaN,NaN,2024-09-26 22:43:52.760000+00:00,service_and_support
440554,WVAQMZM00002061580,[transportation],NaN,NaN,NaN,it has kept me from medical appointments,NaN,NaN,2024-09-26 23:20:28.260000+00:00,service_and_support
440555,WVAQMZM00002121598,"[food, other, ]",NaN,no,NaN,NaN,gas,yes,2023-12-12 19:51:20.214399+00:00,service_and_support


In [8]:
hra_service_and_support[hra_service_and_support['dsnp_member_id']=='101068528400']

,dsnp_member_id,any_concern,losing_house_concern,enough_food,housing_situation,transportation_concern,other_concern,need_help,asad_updated_on_dts,hra_section
5642,101068528400,"[food, housing, ]",no,no,i have housing,NaN,NaN,yes,2024-04-05 19:45:17.030753+00:00,service_and_support


In [10]:
hra_service_and_support["any_concern"].value_counts()

any_concern
[i do not have a problem meeting my needs]                                                                                                         274083
[transportation]                                                                                                                                    18586
[food]                                                                                                                                              16917
[decline]                                                                                                                                           14507
[housing]                                                                                                                                           10694
                                                                                                                                                    ...  
[housing, clothing, child care, transportation, phone, medicine 

In [21]:
unique_elements = set([item for sublist in hra_service_and_support["any_concern"] for item in sublist if item != ''])
unique_elements

{'child care',
 'clothing',
 'decline',
 'food',
 'housing',
 'i do not have a problem meeting my needs',
 'medicine or any health care (medical, dental, mental health, vision)',
 'other',
 'phone',
 'transportation',
 'unable to respond',
 'utilities'}

In [35]:
for element in unique_elements:
    hra_service_and_support['SDOH_' + element.capitalize()] = hra_service_and_support['any_concern'].apply(lambda x: 'Y' if element in x else 'N')


In [36]:
hra_service_and_support.rename(columns = {"SDOH_Medicine or any health care (medical, dental, mental health, vision)":
                                                     "SDOH_Medicine or any health care"}, inplace=True)

In [38]:
sdoh_columns = ["SDOH_Food",
                "SDOH_Housing",
                "SDOH_Utilities",
                "SDOH_Medicine or any health care",
                "SDOH_Child care",
                "SDOH_Clothing",
                "SDOH_Transportation",
                "SDOH_Phone"]
hra_service_and_support[sdoh_columns + ["dsnp_member_id", "any_concern"]]

,SDOH_Food,SDOH_Housing,SDOH_Utilities,SDOH_Medicine or any health care,SDOH_Child care,SDOH_Clothing,SDOH_Transportation,SDOH_Phone,dsnp_member_id,any_concern
0,N,Y,N,N,N,N,N,N,,[housing]
1,N,N,N,N,N,N,N,N,101039985900,[i do not have a problem meeting my needs]
2,N,N,N,N,N,N,N,N,101039986100,[i do not have a problem meeting my needs]
3,N,N,N,N,N,N,N,N,101040086100,[i do not have a problem meeting my needs]
4,N,N,N,N,N,N,N,N,101040408900,[i do not have a problem meeting my needs]
...,...,...,...,...,...,...,...,...,...,...
440552,N,N,N,N,N,N,Y,N,WVAQMZM00001861311,[transportation]
440553,Y,N,Y,N,N,N,N,N,WVAQMZM00001926520,"[utilities, food, ]"
440554,N,N,N,N,N,N,Y,N,WVAQMZM00002061580,[transportation]
440555,Y,N,N,N,N,N,N,N,WVAQMZM00002121598,"[food, other, ]"


In [5]:
hra_service_and_support[hra_service_and_support['dsnp_member_id']=='101773006400']

,dsnp_member_id,any_concern,losing_house_concern,enough_food,housing_situation,transportation_concern,other_concern,need_help,asad_updated_on_dts,hra_section
195078,101773006400,"[transportation, medicine or any health care (...",NaN,no,NaN,"it has kept me from non-medical meetings, appo...",aetna dropped my psychiatrist and now i�m with...,yes,2024-08-07 18:28:06.736666+00:00,service_and_support


In [146]:
#def clean_invalid_utf8(value):
#    if isinstance(value, str):
#        value=re.sub(r'�', '', value)
#    return value
#
#rows = []
#for row in query_job.result():
#    for column, value in row.items():
#        if '�' in value:
#            print(value)
#            cleaned_row = {column:clean_invalid_utf8(value)}
#            print(cleaned_row)
#        else:
#            cleaned_row = {column:value}
#    
#    rows.append(cleaned_row)
#test_df = pd.DataFrame(rows)
#
#test_df = query_job.to_dataframe()